# Conrad et al., 2023 POD Method

Calculate probability of detection using the POD method outlined in Conrad et al., 2023


In [1]:
# First, code for generating wind data - example code
from methods_source import summarize_wind_conditions, calc_avg_windspeed
import datetime

test_1 = datetime.datetime(2022, 10, 10, 18, 16, 10)
test_2 = datetime.datetime(2022, 10, 10, 18, 17, 10)

wind_test1 = summarize_wind_conditions(test_1, test_2)

wind_test2 = calc_avg_windspeed(test_1, direction='forward')

In [2]:
from methods_source import load_overpass_summary, feet_per_meter, calc_avg_windspeed
import pandas as pd
kairos = load_overpass_summary('Kairos', 1, strict_discard=False)
feet_per_meter = feet_per_meter()

kairos_POD_variables = pd.DataFrame()
kairos_POD_variables['overpass_id'] = kairos['overpass_id']
kairos_POD_variables['overpass_datetime'] = kairos['overpass_datetime']
kairos_POD_variables['h'] = kairos['altitude_feet'] / feet_per_meter # flight altitude in m
kairos_POD_variables['u'] = kairos['overpass_datetime'].apply(calc_avg_windspeed) # windspeed in m/s
kairos_POD_variables['Q'] = kairos['release_rate_kgh'] # release rate in kg / hr
kairos_POD_variables['D'] = kairos['operator_detected'].apply(int) # operator detected as 0 or 1

def make_predictor_function(h, u, Q):
    def my_predictor(phi1, phi2, phi3, phi4, phi5, phi6):
        return (phi1 * (Q - phi2) ** phi3) / (((h/1000)**phi4) * ((u - phi5) ** phi6))

    return my_predictor
# Next:
# Generate "my_predictor" for each overpass
# Generate Di ln(Fi) + (1 - Di) ln(1 - Fi) for each overpass
# Sum the column with


In [3]:
print(kairos.operator_detected.apply(int))

0      0
1      0
2      1
3      1
4      1
      ..
344    1
345    1
346    1
347    1
348    1
Name: operator_detected, Length: 349, dtype: int64
